# Tạo **Index JSON** cho **Span Adaptation Vectors** (Train/Test · GramVar/ParaVE)

- **Mục đích**
  - Sinh các file **index `.json` theo từng động từ** để ánh xạ **mỗi span argument (ARG-k)** tới đường dẫn **span vector `.pt`** đã tạo.
  - Dùng cho các bước **truy vấn**, **xây DB tìm kiếm**, **đánh giá** theo argument.

- **Đầu vào**
  - Thư mục **Span AV** (đã sinh từ bước “Span Adaptation Vectors – per-argument”):
    ```
    .../span_adaptation_vectors_{train|test}_{gramvar|parave}/<verb_name>/sentence_<i>_ARG_<k>.pt
    ```
  - Thư mục **dữ liệu gốc** (để lấy `sentence_text` theo chỉ số câu):
    ```
    .../Clean_Dataset/Corpus/Split_{GramVar|ParaVE}/{Train|Test}/*.json
    ```

- **Đầu ra**
  - 4 thư mục index mới (mỗi tổ hợp tập × bộ):
    ```
    .../verb_indexes_span_{train|test}_{gramvar|parave}/<verb_name>.json
    ```
  - Mỗi file `<verb_name>.json` là **một danh sách** (list) các entry, một entry cho **mỗi span argument**:
    ```json
    [
      {
        "sentence_text": "Protein A interacts with protein B.",
        "verb_name": "interact",
        "arg_label": "ARG-0",
        "vector_file_path": "/content/.../span_adaptation_vectors_train_gramvar/interact/sentence_12_ARG_0.pt"
      },
      {
        "sentence_text": "Complex C binds to DNA.",
        "verb_name": "bind",
        "arg_label": "ARG-1",
        "vector_file_path": "/content/.../span_adaptation_vectors_train_gramvar/bind/sentence_7_ARG_1.pt"
      }
    ]
    ```

- **Quy ước tên file span**  
  - Mẫu: `sentence_<i>_ARG_<k>.pt`  
  - Regex trích xuất:  
    ```
    r"sentence_(\d+)_([A-Z]+)_(\d+)\.pt"
    ```
    - group(1) → `i` (chỉ số câu)
    - group(2) → `ARG`
    - group(3) → `k` (số thứ tự argument)
  - **arg_label** được tái tạo: `ARG-k`.

- **Quy trình**
  1. **Quét** tất cả `.pt` trong thư mục Span AV của từng tập/bộ.
  2. **Nhóm theo động từ** (`<verb_name>` là tên thư mục con).
  3. **Xác định file JSON gốc**:
     - Train: `<verb_name>_train_set.json`
     - Test:  `<verb_name>_test_set.json`
  4. **Từ tên file span** suy ra `sentence_idx`, `arg_label`; tra `sentence_text` từ JSON gốc (nếu có).
  5. **Ghi** danh sách entry ra `verb_indexes_span_{train|test}_{gramvar|parave}/<verb_name>.json`.


In [ ]:
# Import các thư viện cần thiết
import os
import json
import glob
from tqdm.auto import tqdm
import re

# THIẾT LẬP CÁC ĐƯỜNG DẪN

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# --- DANH SÁCH CÁC BỘ DỮ LIỆU CẦN TẠO INDEX ---
# Mỗi mục là một tuple chứa: (Tên định danh, Thư mục chứa Span AV, Thư mục dữ liệu gốc, Thư mục output index)
datasets_to_process = [
    # --- Dữ liệu Train ---
    (
        "Train_GramVar",
        os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Train_Content/span_adaptation_vectors_train_gramvar'),
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Train'), # Dữ liệu gốc Train
        os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Train_Content_Index/verb_indexes_span_train_gramvar') # Output index
    ),
    (
        "Train_ParaVE",
        os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Train_Content/span_adaptation_vectors_train_parave'),
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Train'), # Dữ liệu gốc Train
        os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Train_Content_Index/verb_indexes_span_train_parave') # Output index
    ),
    # --- Dữ liệu Test ---
    (
        "Test_GramVar",
        os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Test_Content/span_adaptation_vectors_test_gramvar'),
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Test'), # Dữ liệu gốc Test
        os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Test_Content_Index/verb_indexes_span_test_gramvar') # Output index
    ),
    (
        "Test_ParaVE",
        os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Test_Content/span_adaptation_vectors_test_parave'),
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Test'), # Dữ liệu gốc Test
        os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Test_Content_Index/verb_indexes_span_test_parave') # Output index
    ),
]

# Regex để trích xuất thông tin từ tên file span vector
# Ví dụ: sentence_123_ARG_0.pt -> group(1)=123, group(2)=ARG, group(3)=0
span_filename_pattern = re.compile(r"sentence_(\d+)_([A-Z]+)_(\d+)\.pt")

# HÀM TẠO INDEX

def create_span_indexes(name, span_av_dir, original_data_dir, output_index_dir):
    """
    Tạo các file index .json riêng cho mỗi động từ từ các file span vector.
    """
    print(f"\n=================================================")
    print(f"BẮT ĐẦU TẠO INDEX CHO: {name}")
    print(f"=================================================")

    os.makedirs(output_index_dir, exist_ok=True)

    # Quét tất cả các file .pt trong thư mục span vector
    all_span_files = glob.glob(os.path.join(span_av_dir, '**', '*.pt'), recursive=True)

    if not all_span_files:
        print(f"CẢNH BÁO: Không tìm thấy file span vector nào trong '{span_av_dir}'. Bỏ qua.")
        return

    # Gom nhóm các file theo động từ
    files_by_verb = {}
    for file_path in all_span_files:
        verb_name = os.path.basename(os.path.dirname(file_path))
        if verb_name not in files_by_verb:
            files_by_verb[verb_name] = []
        files_by_verb[verb_name].append(file_path)

    print(f"Tìm thấy {len(all_span_files)} file span vector thuộc {len(files_by_verb)} động từ.")

    pbar_verb = tqdm(total=len(files_by_verb), desc=f"Xử lý động từ {name}", unit="động từ")

    for verb_name, span_files in files_by_verb.items():
        verb_index_list = []
        original_data = None # Cache dữ liệu gốc để đọc nhanh hơn

        # Xác định tên file dữ liệu gốc (.json)
        original_filename_base = f"{verb_name}_train_set.json" if "Train" in name else f"{verb_name}_test_set.json"
        original_data_path = os.path.join(original_data_dir, original_filename_base)

        try:
            # Đọc file dữ liệu gốc một lần cho mỗi động từ
            with open(original_data_path, 'r', encoding='utf-8') as f:
                original_data = json.load(f)
        except FileNotFoundError:
            pbar_verb.write(f"  LỖI: Không tìm thấy file dữ liệu gốc: {original_data_path}. Sẽ thiếu sentence_text.")
        except Exception as e:
             pbar_verb.write(f"  LỖI khi đọc file gốc {original_data_path}: {e}")

        for file_path in span_files:
            match = span_filename_pattern.search(os.path.basename(file_path))
            if not match:
                # pbar_verb.write(f"  CẢNH BÁO: Tên file không đúng định dạng: {os.path.basename(file_path)}")
                continue

            sentence_idx = int(match.group(1))
            arg_type = match.group(2)
            arg_num = match.group(3)
            arg_label = f"{arg_type}-{arg_num}" # tạo lại ARG-X

            sentence_text = "N/A"
            if original_data and sentence_idx < len(original_data):
                sentence_text = original_data[sentence_idx].get('text', 'N/A')

            index_entry = {
                "sentence_text": sentence_text,
                "verb_name": verb_name,
                "arg_label": arg_label,
                "vector_file_path": file_path # Lưu đường dẫn tuyệt đối
            }
            verb_index_list.append(index_entry)

        # Lưu file index cho động từ này
        if verb_index_list:
            output_index_path = os.path.join(output_index_dir, f"{verb_name}.json")
            with open(output_index_path, 'w', encoding='utf-8') as f:
                json.dump(verb_index_list, f, indent=4, ensure_ascii=False)

        pbar_verb.update(1)

    pbar_verb.close()
    print(f" -> Hoàn tất tạo index cho {len(files_by_verb)} động từ.")


# --- 3. THỰC THI CHƯƠNG TRÌNH ---
if __name__ == "__main__":
    for name, span_dir, orig_dir, out_dir in datasets_to_process:
        create_span_indexes(name, span_dir, orig_dir, out_dir)

    print("\n--- QUÁ TRÌNH TẠO INDEX CHO SPAN VECTOR ĐÃ HOÀN TẤT! ---")



BẮT ĐẦU TẠO INDEX CHO: Train_GramVar
Tìm thấy 10390 file span vector thuộc 35 động từ.


Xử lý động từ Train_GramVar:   0%|          | 0/35 [00:00<?, ?động từ/s]

 -> Hoàn tất tạo index cho 35 động từ.

BẮT ĐẦU TẠO INDEX CHO: Train_ParaVE
Tìm thấy 5131 file span vector thuộc 35 động từ.


Xử lý động từ Train_ParaVE:   0%|          | 0/35 [00:00<?, ?động từ/s]

 -> Hoàn tất tạo index cho 35 động từ.

BẮT ĐẦU TẠO INDEX CHO: Test_GramVar
Tìm thấy 1711 file span vector thuộc 35 động từ.


Xử lý động từ Test_GramVar:   0%|          | 0/35 [00:00<?, ?động từ/s]

 -> Hoàn tất tạo index cho 35 động từ.

BẮT ĐẦU TẠO INDEX CHO: Test_ParaVE
Tìm thấy 699 file span vector thuộc 35 động từ.


Xử lý động từ Test_ParaVE:   0%|          | 0/35 [00:00<?, ?động từ/s]

 -> Hoàn tất tạo index cho 35 động từ.

--- QUÁ TRÌNH TẠO INDEX CHO SPAN VECTOR ĐÃ HOÀN TẤT! ---


- **Cấu trúc thư mục (minh hoạ)**
  ```bash
  .../span_adaptation_vectors_train_gramvar/
  └── interact/
      ├── sentence_0_ARG_0.pt
      ├── sentence_0_ARG_1.pt
      └── sentence_1_ARG_0.pt

  .../verb_indexes_span_train_gramvar/
  └── interact.json